In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForQuestionAnswering, pipeline
import torch
import time


In [13]:
# Load the BERT-based uncased model
bert_tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
bert_model = AutoModelForQuestionAnswering.from_pretrained("google-bert/bert-base-uncased")

# Load the Llama model
llama_model_name = "meta-llama/Meta-Llama-3.1-8B"
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_name)
llama_model = AutoModelForCausalLM.from_pretrained(llama_model_name)


# Load SOLAR model
solar_model_name = "upstage/SOLAR-10.7B-v1.0"
solar_tokenizer = AutoTokenizer.from_pretrained(solar_model_name)
solar_model = AutoModelForCausalLM.from_pretrained(solar_model_name)


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.71s/it]


In [14]:
question = "What is the best soil type for growing cassava?"

context = "Cassava grows best in well-drained, sandy-loamy soil."

# Prepare inputs for each model
inputs_bert = bert_tokenizer(question, context, return_tensors="pt")

# Input for Llama 2 (it's a causal language model, only needs the question)
inputs_llama = llama_tokenizer(question, return_tensors="pt")


# Input for SOLAR (like Llama, only needs the question)
inputs_solar = solar_tokenizer(question, return_tensors="pt")



In [15]:
# Get the model predictions (use torch.no_grad to avoid unnecessary gradient computations)
with torch.no_grad():
    # For BERT and AgriBERT (both use question-answering heads)
    bert_output = bert_model(**inputs_bert)
       
    # For Llama and SOLAR (both are causal language models)
    llama_output = llama_model.generate(**inputs_llama, max_length=128)
    solar_output = solar_model.generate(**inputs_solar, max_length=128)
    #llama2_output = llama2_model.generate(**inputs_solar, max_length=128)
    
# Function to extract the predicted answer from BERT-based models
def extract_answer_qa(tokenizer, inputs, outputs):
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(
        tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end])
    )
    return answer

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
import time

In [17]:
# Extracting answers for BERT and AgriBERT models
start_time = time.time()  # Start the timer
bert_answer = extract_answer_qa(bert_tokenizer, inputs_bert, bert_output)
bert_time = time.time() - start_time

print(f"BERT Answer: {bert_answer} (Time taken: {bert_time:.4f} seconds)")

BERT Answer: ##ava? [SEP] cassava grows best in well - drained (Time taken: 0.0010 seconds)


In [18]:
# Extracting the generated text from Llama and SOLAR
start_time = time.time()  # Start the timer
llama_answer = llama_tokenizer.decode(llama_output[0], skip_special_tokens=True)
llama_time = time.time() - start_time  # Calculate time taken
print(f"Llama Answer: {llama_answer} (Time taken: {llama_time:.4f} seconds)")

Llama Answer: What is the best soil type for growing cassava? The best soil for cassava is well-drained, fertile, and slightly acidic. Cassava plants prefer a soil pH between 5.5 and 6.5.
Cassava is a root crop that is native to South America. It is a staple food in many countries and is used to make flour, bread, and other foods. Cassava is a very versatile crop and can be grown in a variety of soil types.
However, the best soil for cassava is well-drained, fertile, and slightly acidic. Cassava plants prefer a soil pH between  (Time taken: 0.0010 seconds)


In [19]:
# Timer and extraction for SOLAR
start_time = time.time()  # Start the timer
solar_answer = solar_tokenizer.decode(solar_output[0], skip_special_tokens=True)
solar_time = time.time() - start_time  # Calculate time taken
print(f"SOLAR Answer: {solar_answer} (Time taken: {solar_time:.4f} seconds)")

SOLAR Answer: What is the best soil type for growing cassava?
What is the best soil type for growing cassava? Cassava is a tropical plant that grows best in well-drained, sandy soils. It can also grow in clay soils, but the soil must be well-drained. Cassava does not like wet soils.
What is the best soil type for growing cassava? Cassava is a tropical plant that grows best in well-drained, sandy soils. It can also grow in clay soils, but the soil must be well-drained. Cassava (Time taken: 0.0010 seconds)


In [20]:
# Display the results
print(f"BERT Answer: {bert_answer}")
#print(f"AgriBERT Answer: {agribert_answer}")
#print(f"Llama2 Answer: {llama2_answer}")
print(f"Llama Answer: {llama_answer}")
print(f"SOLAR Answer: {solar_answer}")


# Display total time taken by all models
total_time = bert_time + llama_time + solar_time
print(f"Total time taken by all models: {total_time:.4f} seconds")

BERT Answer: ##ava? [SEP] cassava grows best in well - drained
Llama Answer: What is the best soil type for growing cassava? The best soil for cassava is well-drained, fertile, and slightly acidic. Cassava plants prefer a soil pH between 5.5 and 6.5.
Cassava is a root crop that is native to South America. It is a staple food in many countries and is used to make flour, bread, and other foods. Cassava is a very versatile crop and can be grown in a variety of soil types.
However, the best soil for cassava is well-drained, fertile, and slightly acidic. Cassava plants prefer a soil pH between 
SOLAR Answer: What is the best soil type for growing cassava?
What is the best soil type for growing cassava? Cassava is a tropical plant that grows best in well-drained, sandy soils. It can also grow in clay soils, but the soil must be well-drained. Cassava does not like wet soils.
What is the best soil type for growing cassava? Cassava is a tropical plant that grows best in well-drained, sandy soil

In [21]:
import numpy as np
from openai import OpenAI


# Initialize the client (using your OpenAI key for SOLAR embeddings)
client = OpenAI(
    api_key="up_SXUJqEaZaGAxC9NCoottzJc8IvPuK",
    base_url="https://api.upstage.ai/v1/solar"
)

# Define the ground truth
ground_truth = "Well-drained sandy loam. The best soil type for growing cassava is well-drained sandy loam"

# Define the answers from BERT, LLaMA, and SOLAR
bert_answer = "growing cassava? [SEP] cassava grows best in well - drained, sandy"
llama_answer = "Cassava is a tropical root crop that is grown in many parts of the world. It is a staple food for millions of people, and it is also used as a source of animal feed and industrial starch. Cassava is a very versatile crop, and it can be grown in a variety of soil types. However, some soil types are better suited for growing cassava than others. The best soil type for growing cassava is a sandy loam. "
solar_answer = "What is the best soil type for growing cassava? Cassava is a tropical plant that grows best in well-drained, sandy soils. It can also grow in clay soils, but the soil must be well-drained. Cassava does not like wet soils.Cassava is a tropical plant that grows best in well-drained, sandy soils. It can also grow in clay soils, but the soil must be well-drained."

# Function to compute embeddings
def get_embedding(text, model_name):
    result = client.embeddings.create(
        model=model_name,
        input=text
    ).data[0].embedding
    return np.array(result)

# Compute embeddings for the query and each answer
query_embedding = get_embedding("What is the best soil type for growing cassava?", "solar-embedding-1-large-query")
ground_truth_embedding = get_embedding(ground_truth, "solar-embedding-1-large-passage")
bert_embedding = get_embedding(bert_answer, "solar-embedding-1-large-passage")
llama_embedding = get_embedding(llama_answer, "solar-embedding-1-large-passage")
solar_embedding = get_embedding(solar_answer, "solar-embedding-1-large-passage")

# Function to compute cosine similarity
def compute_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

# Calculate similarity between the query and the ground truth
ground_truth_similarity = compute_similarity(query_embedding, ground_truth_embedding)

# Calculate similarity between the answers and the ground truth
bert_similarity = compute_similarity(ground_truth_embedding, bert_embedding)
llama_similarity = compute_similarity(ground_truth_embedding, llama_embedding)
solar_similarity = compute_similarity(ground_truth_embedding, solar_embedding)

# Print the similarity results
print(f"Ground Truth Similarity: {ground_truth_similarity}")
print(f"BERT Similarity: {bert_similarity}")
print(f"LLaMA Similarity: {llama_similarity}")
print(f"SOLAR Similarity: {solar_similarity}")

# Determine which model provides the best answer
best_model = max(bert_similarity, llama_similarity, solar_similarity)
if best_model == bert_similarity:
    print("BERT provided the closest answer.")
elif best_model == llama_similarity:
    print("LLaMA provided the closest answer.")
else:
    print("SOLAR provided the closest answer.")

Ground Truth Similarity: 0.6932431890256054
BERT Similarity: 0.8252637818874504
LLaMA Similarity: 0.839351073840357
SOLAR Similarity: 0.9200447754030694
SOLAR provided the closest answer.


In [22]:
import numpy as np
from openai import OpenAI
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the client (using your OpenAI key for SOLAR embeddings)
client = OpenAI(
    api_key="up_SXUJqEaZaGAxC9NCoottzJc8IvPuK",
    base_url="https://api.upstage.ai/v1/solar"
)

# Define the ground truth
ground_truth = "Well-drained sandy loam. The best soil type for growing cassava is well-drained sandy loam"

# Define the answers from BERT, LLaMA, and SOLAR
bert_answer = "growing cassava? [SEP] cassava grows best in well - drained, sandy"
llama_answer = "Cassava is a tropical root crop that is grown in many parts of the world. It is a staple food for millions of people, and it is also used as a source of animal feed and industrial starch. Cassava is a very versatile crop, and it can be grown in a variety of soil types. However, some soil types are better suited for growing cassava than others. The best soil type for growing cassava is a sandy loam. "
solar_answer = "What is the best soil type for growing cassava? Cassava is a tropical plant that grows best in well-drained, sandy soils. It can also grow in clay soils, but the soil must be well-drained. Cassava does not like wet soils.Cassava is a tropical plant that grows best in well-drained, sandy soils. It can also grow in clay soils, but the soil must be well-drained."

# Function to compute embeddings
def get_embedding(text, model_name):
    result = client.embeddings.create(
        model=model_name,
        input=text
    ).data[0].embedding
    return np.array(result)

# Compute embeddings for the query and each answer
query_embedding = get_embedding("What is the best soil type for growing cassava?", "solar-embedding-1-large-query")
ground_truth_embedding = get_embedding(ground_truth, "solar-embedding-1-large-passage")
bert_embedding = get_embedding(bert_answer, "solar-embedding-1-large-passage")
llama_embedding = get_embedding(llama_answer, "solar-embedding-1-large-passage")
solar_embedding = get_embedding(solar_answer, "solar-embedding-1-large-passage")

# Function to compute cosine similarity
def compute_similarity(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))

# Calculate similarity between the query and the ground truth
ground_truth_similarity = compute_similarity(query_embedding, ground_truth_embedding)

# Calculate similarity between the answers and the ground truth
bert_similarity = compute_similarity(ground_truth_embedding, bert_embedding)
llama_similarity = compute_similarity(ground_truth_embedding, llama_embedding)
solar_similarity = compute_similarity(ground_truth_embedding, solar_embedding)

# Print the similarity results
print(f"Ground Truth Similarity: {ground_truth_similarity}")
print(f"BERT Similarity: {bert_similarity}")
print(f"LLaMA Similarity: {llama_similarity}")
print(f"SOLAR Similarity: {solar_similarity}")

# Determine which model provides the best answer
best_model = max(bert_similarity, llama_similarity, solar_similarity)
if best_model == bert_similarity:
    print("BERT provided the closest answer.")
elif best_model == llama_similarity:
    print("LLaMA provided the closest answer.")
else:
    print("SOLAR provided the closest answer.")

Ground Truth Similarity: 0.6932431890256054
BERT Similarity: 0.8252637818874504
LLaMA Similarity: 0.839351073840357
SOLAR Similarity: 0.9200447754030694
SOLAR provided the closest answer.


Similarity between query and document: 0.6706783327422627
